# Neural networks simulation (Synchronization Problem)

This file is going to study any neural netwrok class which is defined in the `<network_reference.py>` file.

In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

In [2]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas() #This line make sure that the progress bars looks natural

## Order your neuron model of interest

In [3]:
current_models = ['IF','Rotational','Non_repulsive_rotational']
neuron_model = current_models[0]

with open("network_reference.py") as net_ref: 
    lines = net_ref.readlines() #read 
 
    #modify 
    lines[0] = "neuron_engine = '{}'\n".format(neuron_model) #you can replace zero with any line number. 
 
with open("network_reference.py", "w") as net_ref: 
    net_ref.writelines(lines) #write back 

In [4]:
from network_reference import Network_of_neurons

## Parameters and constants

In [5]:
num_neurons = 10000
total_time = 100

num_ensembles_each_g = 8

In [6]:
# random_input_span = (9.5,13.5)
random_input_span = (1.2,2.8)

In [7]:
connectivity_min,connectivity_max, connectivity_step = 0, 7.5, 0.3
connectivity_span = np.arange(connectivity_min,connectivity_max,connectivity_step).round(2)

In [8]:
delay_min, delay_max, delay_step =0.1, 0.11, 0.02
delay_span = np.arange(delay_min, delay_max, delay_step).round(2)

In [9]:
alpha_min, alpha_max, alpha_step = 20, 30, 10
alpha_span = np.arange(alpha_min, alpha_max, alpha_step).round(2)

## Prepare the storage to record the outputs

In [10]:
version = '_v1.0'
current_size_networks_path = os.path.join(neuron_model + '_ensembles','N{}_T{}_I{}_{}'.format(num_neurons,total_time,random_input_span[0],random_input_span[1]) + version )
try:
    os.makedirs(current_size_networks_path)
except:
    pass

In [11]:
from storage_modifications import *

## Creating various ensembles:

In [12]:
for connectivity in tqdm( connectivity_span , desc='Variation of connectivities'):
    group_folder_name = 'g_{}'.format(connectivity)
    group_storage = make_inner_dir(current_size_networks_path, group_folder_name)
    
    for delay in tqdm( delay_span , desc='Variation of delay'):
        d_subgroup_folder_name = 'd_{}'.format(delay)
        d_subgroup_storage = make_inner_dir(group_storage, d_subgroup_folder_name)
        
        for alpha in tqdm( alpha_span , desc='Variation of alpha') :
            alpha_subgroup_folder_name = 'alpha_{}'.format(alpha)
            alpha_subgroup_storage = make_inner_dir(d_subgroup_storage, alpha_subgroup_folder_name)
            last_index = last_ensemble_index_in_group(alpha_subgroup_storage)

            for i in tqdm( range(num_ensembles_each_g) , desc='Making ensembles'):
                sample_model = Network_of_neurons(num_neurons=num_neurons,
                                                  g=connectivity, alpha = alpha)
                sample_model.ignite(random_input_span, total_time = total_time,
                                    time_step = 0.01, delay_time = delay)

                current_ensemble_folder = make_inner_dir(alpha_subgroup_storage, str(last_index + i + 1) )

                save_sigma(sample_model.report_sigma(), current_ensemble_folder)
                save_field_period(*sample_model.report_e_period_fft(), current_ensemble_folder)
                save_field_average(sample_model.e_arr, current_ensemble_folder)
                
                if neuron_model in current_models[1:]: save_amin_saman_param(sample_model.report_sync_parameter(), current_ensemble_folder)
                save_mean_spiking_periods( sample_model.report_spikies_period_mean(), current_ensemble_folder )

C:\Users\mohsen\AppData\Local\Temp/ipykernel_6100/1212606915.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for connectivity in tqdm( connectivity_span , desc='Variation of connectivities'):


Variation of connectivities:   0%|          | 0/25 [00:00<?, ?it/s]

C:\Users\mohsen\AppData\Local\Temp/ipykernel_6100/1212606915.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for delay in tqdm( delay_span , desc='Variation of delay'):


Variation of delay:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\mohsen\AppData\Local\Temp/ipykernel_6100/1212606915.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for alpha in tqdm( alpha_span , desc='Variation of alpha') :


...

Making ensembles:   0%|          | 0/8 [00:00<?, ?it/s]





network dynamic:   0%|                                                                        | 0/9999 [00:00<?, ?it/s]



network dynamic:   3%|█▌                                                          | 265/9999 [00:00<00:03, 2649.19it/s]



network dynamic:   5%|███▏                                                        | 530/9999 [00:00<00:03, 2513.06it/s]



network dynamic:   8%|████▋                                                       | 782/9999 [00:00<00:03, 2478.89it/s]



network dynamic:  10%|██████                                                     | 1031/9999 [00:00<00:03, 2473.63it/s]



network dynamic:  13%|███████▌                                                   | 1280/9999 [00:00<00:03, 2479.40it/s]



network dynamic:  15%|█████████                                                  | 1533/9999 [00:00<00:03, 2495.43it/s]



network dynamic:  18%|██████████▌                                                | 1795/9999 [00:00<00:03, 2535.44it/s]



network dyna

network dynamic:   0%|                                                                        | 0/9999 [00:00<?, ?it/s]



network dynamic:   2%|█▍                                                          | 236/9999 [00:00<00:04, 2337.47it/s]



network dynamic:   5%|██▊                                                         | 470/9999 [00:00<00:04, 2263.29it/s]



network dynamic:   7%|████▎                                                       | 724/9999 [00:00<00:03, 2377.62it/s]



network dynamic:  10%|█████▊                                                      | 978/9999 [00:00<00:03, 2440.68it/s]



network dynamic:  12%|███████▎                                                   | 1236/9999 [00:00<00:03, 2489.31it/s]



network dynamic:  15%|████████▊                                                  | 1495/9999 [00:00<00:03, 2523.23it/s]



network dynamic:  18%|██████████▎                                                | 1753/9999 [00:00<00:03, 2541.14it/s]



network dynamic:

network dynamic:  44%|██████████████████████████                                 | 4422/9999 [00:00<00:00, 6018.34it/s]



network dynamic:  50%|█████████████████████████████▋                             | 5025/9999 [00:00<00:00, 6005.49it/s]



network dynamic:  57%|█████████████████████████████████▌                         | 5680/9999 [00:00<00:00, 6171.79it/s]



network dynamic:  63%|█████████████████████████████████████▏                     | 6298/9999 [00:01<00:00, 6168.21it/s]



network dynamic:  69%|████████████████████████████████████████▊                  | 6916/9999 [00:01<00:00, 6122.74it/s]



network dynamic:  75%|████████████████████████████████████████████▍              | 7529/9999 [00:01<00:00, 6104.88it/s]



network dynamic:  81%|████████████████████████████████████████████████           | 8140/9999 [00:01<00:00, 6034.95it/s]



network dynamic:  87%|███████████████████████████████████████████████████▌       | 8744/9999 [00:01<00:00, 5983.27it/s]



network dynamic:

network dynamic:  79%|██████████████████████████████████████████████▎            | 7857/9999 [00:01<00:00, 5813.91it/s]



network dynamic:  85%|█████████████████████████████████████████████████▉         | 8462/9999 [00:01<00:00, 5867.05it/s]



network dynamic:  91%|█████████████████████████████████████████████████████▌     | 9081/9999 [00:02<00:00, 5962.38it/s]



network dynamic: 100%|███████████████████████████████████████████████████████████| 9999/9999 [00:02<00:00, 4598.42it/s]


In [13]:
!ipython nbconvert --to python any_neural_model_ensembles.ipynb

[TerminalIPythonApp] WARNING | Subcommand `ipython nbconvert` is deprecated and will be removed in future versions.
[TerminalIPythonApp] WARNING | You likely want to use `jupyter nbconvert` in the future
[NbConvertApp] Converting notebook any_neural_model_ensembles.ipynb to python
[NbConvertApp] Writing 3936 bytes to any_neural_model_ensembles.py
